# Set up

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
stdsc = StandardScaler()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Read files

In [0]:
from glob import glob
filenames = glob('/content/drive/My Drive/raw data 17-19/SH*.csv')
dataframes = [pd.read_csv(f) for f in filenames]

Set variables

In [0]:
## Set cash variable
cash = 1000000

In [0]:
## Create asset DF
asset_df = pd.DataFrame(columns = {"stock", "purchase_P", "num_share", "current_P", "value"})

Define functions

In [0]:
def addVar(data):
    data["MA5"] = 0.0
    data["MA15"] = 0.0
    data["rLag5"] = 0.0
    data["rLag15"] = 0.0
    data["dailyR"]= 0.0
    data["RSI"] = 0.0
    data["stochK"] = 0.0
    data["stochD"] = 0.0

    for i in range(2, len(data)):
        data.loc[i,"dailyR"] = 100*(data["Close"][i-1]/(data["Close"][i-2]) - 1)

    for i in range(5, len(data)):
        data.loc[i,"MA5"] = data["Close"][i-5:i].mean()
        data.loc[i,"rLag5"] = 100*(data["Close"][i-1] - data["Close"][i-5])/(data["Close"][i-5])

    for i in range(15, len(data)):
        data.loc[i,"MA15"] = data["Close"][i-15:i].mean()
        data.loc[i,"rLag15"] = 100*(data["Close"][i-1] - data["Close"][i-15])/(data["Close"][i-15])

    for i in range(14, len(data)):
        temp = data["dailyR"][i-14:i]
        data.loc[i,"RSI"] = 100-100/(1+(np.mean(temp[temp>0])/-np.mean(temp[temp<0])))
        data.loc[i,"stochK"] = 100*(data["Close"][i-1] - min(data["Low"][i-14:i]))/(max(data["High"][i-14:i])-min(data["Low"][i-14:i]))

    for i in range(17, len(data)):
        data.loc[i,"stochD"] = data["stochK"][i-3:i].mean()
    
    data.fillna(value = 0, inplace= True)
    
    return data

In [0]:
def computeR(data, start):

    X_train = data.iloc[start-365:start,6:]
    X_train_std = pd.DataFrame(stdsc.fit_transform(X_train), columns = data.columns[6:])
    y_train = data.Close[start-365:start].reset_index(drop = True)

    lm = linear_model.LinearRegression()
    model = lm.fit(X_train_std,y_train)

    for end in range(start, start+10):
    
        data = data.append(pd.Series(0, index = data.columns), ignore_index = True)

        data.loc[end,"dailyR"] = 100*(data["Close"][end-1]/(data["Close"][end-2]) - 1)

        data.loc[end,"MA5"] = data["Close"][end-5:end].mean()
        data.loc[end,"rLag5"] = 100*(data["Close"][end-1] - data["Close"][end-5])/(data["Close"][end-5])


        data.loc[end,"MA15"] = data["Close"][end-15:end].mean()
        data.loc[end,"rLag15"] = 100*(data["Close"][end-1] - data["Close"][end-15])/(data["Close"][end-15])


        temp = data["dailyR"][end-14:end]
        data.loc[end,"RSI"] = 100-100/(1+(np.mean(temp[temp>0])/-np.mean(temp[temp<0])))
        data.loc[end,"stochK"] = 100*(data["Close"][end-1] - min(data["Low"][end-14:end]))/(max(data["High"][end-14:end])-min(data["Low"][end-14:end]))

        data.loc[end,"stochD"] = data["stochK"][end-3:end].mean()
        
        data.fillna(value = 0, inplace = True)

        X_test = data.iloc[end,6:].values.reshape(-1,8)
        X_test_std = pd.DataFrame(stdsc.transform(X_test), columns = data.columns[6:])
        y_test = data.Close[end]

        result_pred = model.predict(X_test_std)
        data.Close[end] = result_pred

    R10 = (data["Close"][end]/data["Close"][start]-1)*100
    return R10

In [0]:
def selectStock(df, start, num):
    stock = []
    r = []
    price = []
    risk = []
    
    for s in range(len(df)):
        data = df[s]
        data = data.iloc[:start,]
        data = addVar(data)
        stock.append(s)
        r.append(computeR(data, start))
        price.append(data.loc[start-1,"Close"])
        risk.append(np.std(data.Close[start-365:start]))

    r_df = pd.DataFrame({"stock":stock, "price":price, "return":r, "risk": risk})
    r_df["standard_r"] = r_df["return"]/r_df["risk"]
    r_df = r_df.sort_values("return", ascending = False)
    stocks = r_df.iloc[:num,]
    return stocks

In [0]:
def computeWealth(df,start):
    
    wealth = []

    for day in range(start,start+10):
        for s in df.index:
            df.loc[s,"current_P"] = dataframes[s].loc[day,"Close"]
            df.loc[s,"value"] = df.loc[s, "num_share"] * df.loc[s, "current_P"]
        wealth.append(cash + sum(asset_df.value))
    return wealth

# Initial run

In [0]:
## Day 505 (index 504) purchase brand new 10 stocks
stocks = selectStock(dataframes, 504, 10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [0]:
stocks

,stock,price,return,risk,standard_r
15,15,218.2768,15.258642,20.837961,0.732252
11,11,187.8494,15.086296,50.631972,0.297960
25,25,83.5914,10.958313,9.929056,1.103661
2,2,13.2047,10.732944,3.917856,2.739494
47,47,5.5667,10.562243,1.385063,7.625822
14,14,43.2765,10.378977,24.151567,0.429743
12,12,64.8743,9.069262,15.740811,0.576162
6,6,7.4681,7.627333,0.969470,7.867532
24,24,585.9996,7.304624,101.981878,0.071627
49,49,7.2210,5.797944,2.202433,2.632518


In [0]:
for s in stocks["stock"]:
    purchase_p = stocks.loc[s,"price"]
    num_share = (cash/len(stocks))/(stocks.loc[s,"price"]*(1+0.00065))
    current_p = purchase_p
    value = num_share * purchase_p

    asset_df = asset_df.append({"stock":int(s), "purchase_P":purchase_p, "num_share":num_share,"current_P": current_p, "value":value}
                               , ignore_index = True)  
    asset_df.stock = asset_df.stock.astype(int)
asset_df = asset_df.set_index("stock")
cash -= sum(asset_df.value) * (1+0.00065)
asset_df

,value,purchase_P,num_share,current_P
stock,,,,
15,99935.042223,218.2768,457.836299,218.2768
11,99935.042223,187.8494,531.995536,187.8494
25,99935.042223,83.5914,1195.518226,83.5914
2,99935.042223,13.2047,7568.141815,13.2047
47,99935.042223,5.5667,17952.295296,5.5667
14,99935.042223,43.2765,2309.221915,43.2765
12,99935.042223,64.8743,1540.441164,64.8743
6,99935.042223,7.4681,13381.588653,7.4681
24,99935.042223,585.9996,170.537731,585.9996


In [0]:
wealth = computeWealth(asset_df,504)
wealth

[985107.5462918258,
 992570.4773781737,
 988669.5435809135,
 986345.7928023863,
 985927.849793247,
 977442.6982762639,
 974556.5112539545,
 964719.8126531582,
 965702.2647236327,
 968928.0633684692]

# Recursive Run

In [0]:
start = 514

while start < 744:
    
    stocks = selectStock(dataframes, start, 10)
    
    stock_buy=[i for i in stocks.stock if i not in asset_df.index]
    stock_sell=[i for i in asset_df.index if i not in stocks.stock]
    stock_keep=[i for i in stocks.stock if i in asset_df.index]

    for s in stock_sell:
        cash += asset_df.loc[s,"num_share"] * asset_df.loc[s, "current_P"] * (1-0.00065)
        asset_df = asset_df.drop(s)

    for b in stock_buy:
        purchase_p = stocks.loc[b,"price"]
        num_share = (cash/len(stock_buy))/(stocks.loc[b,"price"]*(1+0.00065))
        current_p = purchase_p
        value = num_share * purchase_p

        asset_df.loc[b,"value"] = value
        asset_df.loc[b,"purchase_P"] = purchase_p
        asset_df.loc[b,"num_share"] = num_share
        asset_df.loc[b,"current_P"] = current_p

    cash -= sum(asset_df.loc[i,"purchase_P"]*asset_df.loc[i,"num_share"] * (1+0.00065) for i in stock_buy)
    
    wealth.append(computeWealth(asset_df,start))
    
    start = start + 10
    

Streaming output truncated to the last 5000 lines.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set 

In [0]:
wealth

[985107.5462918258,
 992570.4773781737,
 988669.5435809135,
 986345.7928023863,
 985927.849793247,
 977442.6982762639,
 974556.5112539545,
 964719.8126531582,
 965702.2647236327,
 968928.0633684692,
 [972247.216944368,
  961396.2306324383,
  976438.1909437056,
  1006609.2439982259,
  1016422.1318258069,
  1017852.5540761796,
  1027223.3789931458,
  1013107.7333286783,
  1017431.9964346476,
  1010221.4942527632],
 [1020654.3725639337,
  1024085.5741355526,
  1023843.6122950737,
  1043243.0495173887,
  1054594.4223268412,
  1042149.4264162749,
  1038349.3941329108,
  1039867.5399224171,
  1048856.5582985233,
  1050806.373291611],
 [1042108.60919885,
  1035596.8191816618,
  1040456.6790356543,
  1055004.878152879,
  1070026.566352545,
  1074318.4558790694,
  1103882.11418594,
  1102608.2887902565,
  1086574.3708927415,
  1116229.864825156],
 [1116612.2396914063,
  1130609.4631645735,
  1125813.6495766724,
  1133907.6928992576,
  1185477.6846625214,
  1194523.3774260338,
  1190439.55585119

# Last run

In [0]:
cash = 0

In [0]:
stocks = selectStock(dataframes, 744, 10)
  
stock_buy=[i for i in stocks.stock if i not in asset_df.index]
stock_sell=[i for i in asset_df.index if i not in stocks.stock]
stock_keep=[i for i in stocks.stock if i in asset_df.index]

for s in stock_sell:
    cash += asset_df.loc[s,"num_share"] * asset_df.loc[s, "current_P"] * (1-0.00065)
    asset_df = asset_df.drop(s)

for b in stock_buy:
    purchase_p = stocks.loc[b,"price"]
    num_share = (cash/len(stock_buy))/(stocks.loc[b,"price"]*(1+0.00065))
    current_p = purchase_p
    value = num_share * purchase_p

    asset_df.loc[b,"value"] = value
    asset_df.loc[b,"purchase_P"] = purchase_p
    asset_df.loc[b,"num_share"] = num_share
    asset_df.loc[b,"current_P"] = current_p

cash -= sum(asset_df.loc[i,"purchase_P"]*asset_df.loc[i,"num_share"] * (1+0.00065) for i in stock_buy)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [0]:
money = []

for day in range(744,756):
    for s in asset_df.index:
        asset_df.loc[s,"current_P"] = dataframes[s].loc[day,"Close"]
        asset_df.loc[s,"value"] = asset_df.loc[s, "num_share"] * asset_df.loc[s, "current_P"]
    money.append(cash + sum(asset_df.value))

for s in asset_df.index:
  cash += asset_df.loc[s, "num_share"] * dataframes[s].loc[755,"Close"]*(1-0.00065)
money.append(cash)

In [0]:
money

[1120082.2225736047,
 1114489.3110889834,
 1129957.3357080768,
 1141842.902487097,
 1156350.833136748,
 1153546.953062829,
 1154691.2158433981,
 1160670.2065999196,
 1148606.867556598,
 1152773.0164054,
 1153556.9057839832,
 1165746.6689513025,
 1164988.933616484]